In [60]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
import os
import numpy as np
FOLDER_NAME = 'dataset_extracted'


In [61]:
# Create label map
label_map = {}
for (root, folders, files) in os.walk(FOLDER_NAME):
    if root == FOLDER_NAME:
        for index, foldername in enumerate(folders):
            label_map[foldername] = index


print(label_map)


{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}


In [62]:
sequence, label = [], []
target_length = 48
for (root, folders, files) in os.walk(FOLDER_NAME):
    # if root != FOLDER_NAME:
    for filename in files:
        file_path = os.path.join(os.path.relpath(
            root, FOLDER_NAME), filename)
        res = np.load(f'{FOLDER_NAME}/{file_path}')
        for _ in range(target_length-res.shape[0]):
            res = np.vstack((res, res[-1, :]))
        sequence.append(np.array(res))
        label.append(label_map[os.path.basename(root)])

print(np.array(sequence).shape)
print(np.array(label).shape)


(1690, 48, 1662)
(1690,)


In [76]:
sequence = np.concatenate([sequence, sequence, sequence], axis=0)
label = np.concatenate([label, label, label], axis=0)

print(np.array(sequence).shape)
print(np.array(label).shape)

(5070, 48, 1662)
(5070,)


In [77]:
X_train, X_test, y_train, y_test = train_test_split(np.array(sequence), keras.utils.to_categorical(
    np.array(label).astype(int), num_classes=26, dtype='float32'))

print(X_train.shape, X_test.shape)


(3802, 48, 1662) (1268, 48, 1662)


In [65]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard

In [66]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)
CLASSES = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
           'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [71]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(48,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(np.array(CLASSES).shape[0], activation='softmax'))

In [72]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

In [ ]:
model.summary()